## 6. Agent : PropositionalLogicAgent (Definitions)

Cet agent est specialise dans l'analyse logique formelle (logique propositionnelle) en utilisant la bibliotheque Java Tweety via JPype.

**Role :**
*   Traduire des extraits de texte/arguments en un "Belief Set" PL (format Tweety) (`PLAnalyzer.semantic_TextToPLBeliefSet`, puis `StateManager.add_belief_set`).
*   Generer des requetes logiques pertinentes pour interroger un Belief Set (`PLAnalyzer.semantic_GeneratePLQueries`).
*   Executer ces requetes via la fonction native interfacant avec Tweety (`PLAnalyzer.execute_and_log_pl_query` - **RECOMMANDE** car logging automatique, ou `PLAnalyzer.execute_pl_query` + `StateManager.log_query_result`). **Necessite une JVM prete.**
*   Interpreter les resultats bruts des requetes en langage naturel (`PLAnalyzer.semantic_InterpretPLResult`).
*   Repondre aux taches assignees par le PM (`StateManager.add_answer`).

**Composants Definis Ci-dessous :**
*   `PropositionalLogicPlugin` (Classe V11 - avec logging automatique)
*   Prompts Semantiques (`prompt_text_to_pl_v8`, `prompt_gen_pl_queries_v8`, `prompt_interpret_pl_v8`) et Fonction Setup (`setup_pl_kernel`)
*   Instructions Systeme (`PL_AGENT_INSTRUCTIONS` - V12)

**Syntaxe Cruciale :** Le Belief Set et les requetes doivent respecter la BNF du `PlParser` de Tweety. L'operateur `>>` est a eviter.

**Nouveaute V11:** La fonction `execute_and_log_pl_query` combine execution ET logging pour garantir que les resultats apparaissent dans la cross-validation finale.

# Notebook 2 : Definition de l'Agent de Logique Propositionnelle (PL)

## Introduction

Ce notebook fait partie d'une architecture multi-agents pour l'analyse d'arguments. Il definit completement le **PropositionalLogicAgent**, un agent specialise dans la traduction et l'analyse formelle d'arguments en logique propositionnelle.

### Objectifs pedagogiques

A la fin de ce notebook, vous comprendrez:

1. **Architecture d'un agent specialise** - Structure complete d'un agent avec plugin natif, fonctions semantiques, et instructions
2. **Integration Java/Python** - Utilisation de JPype pour interfacer avec la bibliotheque Tweety (Java) depuis Python
3. **Logique propositionnelle formelle** - Syntaxe BNF de Tweety et operateurs logiques
4. **Semantic Kernel avance** - Configuration de prompts avec gestion de contenu potentiellement dangereux

### Pre-requis

Ce notebook suppose que vous avez execute:
- **Notebook 0** : Configuration de l'environnement (JVM, Tweety, OpenAI API)
- **Notebook 1** : Definition de l'InformalAnalysisAgent

### Structure du notebook

| Section | Contenu |
|---------|---------|
| 6. Agent PL - Introduction | Vue d'ensemble du role et des composants |
| 6.1. Classe Plugin | `PropositionalLogicPlugin` avec methodes natives Java |
| 6.2. Prompts Semantiques | Templates pour traduction, generation de requetes, interpretation |
| 6.3. Instructions Systeme | Regles strictes de comportement de l'agent |
| 6.4. Test (commente) | Cellule de test du plugin (optionnelle) |

---

### 🔌 Classe Plugin : PropositionalLogicPlugin

## Partie 1 : Plugin Natif - Interface Java/Python avec Tweety

### Contexte : Pourquoi un plugin natif ?

L'agent de logique propositionnelle doit effectuer des operations formelles complexes :
- **Parser** des formules en syntaxe Tweety (grammaire BNF stricte)
- **Raisonner** sur des belief sets avec un moteur SAT
- **Executer** des requetes logiques et retourner des resultats formels

Ces operations sont implementees dans la bibliotheque Java **TweetyProject**. Plutot que de reimplementer cette logique complexe en Python, nous utilisons **JPype** pour appeler directement le code Java depuis Python.

### Architecture du PropositionalLogicPlugin

Le plugin agit comme une **facade** entre le Semantic Kernel (Python) et Tweety (Java):

```
Semantic Kernel (Python)
         |
         v
PropositionalLogicPlugin
         |
    +----+----+
    |         |
    v         v
Methodes      Methodes
Semantiques   Natives
(LLM)         (JPype -> Java)
         |
         v
    TweetyProject
    (PlParser, SatReasoner)
```

### Fonctionnalites cles

| Methode | Type | Description |
|---------|------|-------------|
| `execute_pl_query` | Native | Execute une requete PL (ancienne version) |
| `execute_and_log_pl_query` | Native | **RECOMMANDEE** - Execute ET logue automatiquement |
| `_internal_parse_formula` | Interne | Parse une formule avec PlParser |
| `_internal_parse_belief_set` | Interne | Parse un belief set complet |
| `_internal_execute_query` | Interne | Execute une requete avec SatReasoner |

### Nouveaute V12 : Logging automatique

La version V12 introduit un mecanisme de **logging automatique** crucial pour la cross-validation finale:

1. **Log interne** - Chaque requete est enregistree dans `_query_log` (liste Python)
2. **Callback externe** - Si configure, appelle automatiquement `StateManager.log_query_result()`
3. **Garantie de trace** - Assure que toutes les requetes apparaissent dans le rapport final

> **Note technique** : Le callback est configure lors de `setup_pl_kernel()` en passant une reference a `StateManagerPlugin.log_query_result`.

---

In [ ]:
# %% CELLULE [6.1] - Classe PropositionalLogicPlugin
# (Remplace une partie de l'ancienne cellule 7bbc31e5)
# V12 - Fix callback parameters to match StateManagerPlugin.log_query_result

import logging
import jpype
from typing import List, Optional, Any, Dict, Callable
from semantic_kernel.functions import kernel_function
import time
from datetime import datetime

logger = logging.getLogger("Orchestration.AgentPL.Defs")
plugin_logger = logging.getLogger("Orchestration.PLAnalyzerPlugin")

# --- Configuration Logger Plugin ---
if not plugin_logger.handlers and not plugin_logger.propagate:
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); plugin_logger.addHandler(handler); plugin_logger.setLevel(logging.INFO)

# --- Plugin Spécifique PLAnalyzer (V12 - Fix callback params) ---
class PropositionalLogicPlugin:
    """
    Plugin SK pour l'analyse en logique propositionnelle via Tweety/JPype.
    Contient la logique d'interaction Java et expose une fonction native au kernel.
    V12: Fix callback parameters to match StateManagerPlugin.log_query_result(belief_set_id, query, raw_result).
    """
    _logger: logging.Logger
    _jvm_ok: bool
    _PlParser: Optional[Any] = None
    _SatReasoner: Optional[Any] = None
    _PlFormula: Optional[Any] = None
    _parser_instance: Optional[Any] = None
    _reasoner_instance: Optional[Any] = None
    _log_callback: Optional[Callable] = None  # Callback vers StateManager.log_query_result
    _query_log: List[Dict] = None  # Log interne des requêtes pour cross-validation

    def __init__(self, log_callback: Optional[Callable] = None):
        self._logger = plugin_logger
        self._jvm_ok = False
        self._log_callback = log_callback
        self._query_log = []  # Initialiser le log interne
        self._logger.info("Instance PropositionalLogicPlugin créée (V12 avec callback fix).")
        self._initialize_jvm_components()

    def set_log_callback(self, callback: Callable):
        """Configure le callback pour logger automatiquement les résultats de requêtes."""
        self._log_callback = callback
        self._logger.info("Callback de logging configuré pour PropositionalLogicPlugin.")

    def get_query_log(self) -> List[Dict]:
        """Retourne le log interne des requêtes exécutées."""
        return self._query_log.copy()

    def _initialize_jvm_components(self):
        """Vérifie la JVM et charge les classes/instances Tweety."""
        global jvm_ready # Vérifier la variable globale définie dans la cellule JPype
        if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
            self._logger.info("JVM détectée comme prête. Chargement classes Tweety...")
            self._jvm_ok = True
        else:
            self._logger.critical("JVM non prête lors init Plugin PL! Fonctions natives échoueront.")
            self._jvm_ok = False
            return

        try:
            self._PlParser = jpype.JClass("org.tweetyproject.logics.pl.parser.PlParser")
            self._SatReasoner = jpype.JClass("org.tweetyproject.logics.pl.reasoner.SatReasoner")
            self._PlFormula = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")
            self._parser_instance = self._PlParser()
            self._reasoner_instance = self._SatReasoner()
            self._logger.info("Classes et instances Java Tweety chargées.")
        except Exception as e:
            self._logger.critical(f"Erreur chargement classes/instances Tweety: {e}", exc_info=True)
            self._jvm_ok = False

    # --- Méthodes Internes ---
    def _internal_parse_formula(self, formula_string: str) -> Optional[Any]:
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse formula: JVM/Parser non prêt ('{formula_string[:60]}...').")
            return None
        try:
            self._logger.debug(f"Parsing formule: '{formula_string}'")
            return self._parser_instance.parseFormula(formula_string)
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing formule '{formula_string}': {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Formule: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing formule '{formula_string}': {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Formule: {e}") from e

    def _internal_parse_belief_set(self, belief_set_string: str) -> Optional[Any]:
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse BS: JVM/Parser non prêt (BS: '{belief_set_string[:60]}...').")
            return None
        try:
            belief_set_string_cleaned = belief_set_string.replace("\\\\n", "\\n")
            self._logger.debug(f"Parsing belief set (nettoyé): '{belief_set_string_cleaned[:100]}...'")
            parsed_bs = self._parser_instance.parseBeliefBase(belief_set_string_cleaned)
            self._logger.debug(f" -> BS parsé (type: {type(parsed_bs)}, taille: {parsed_bs.size()}).")
            return parsed_bs
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing BS (extrait: '{belief_set_string[:60]}...'): {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Belief Set: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing BS: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Belief Set: {e}") from e

    def _internal_execute_query(self, belief_set_obj: Any, formula_obj: Any) -> Optional[bool]:
        if not self._jvm_ok or not self._reasoner_instance or not self._PlFormula:
            self._logger.error("Execute query: JVM/Reasoner/Formula non prêt.")
            return None
        try:
            if not isinstance(formula_obj, self._PlFormula):
                 raise TypeError(f"Objet requête n'est pas un PlFormula (type: {type(formula_obj)})")
            self._logger.debug(f"Exécution requête avec raisonneur '{self._reasoner_instance.getClass().getName()}'...")
            result_java_boolean = self._reasoner_instance.query(belief_set_obj, formula_obj)
            self._logger.debug(f" -> Résultat brut Java: {result_java_boolean}")
            if result_java_boolean is None:
                self._logger.warning("Requête Tweety a retourné null (indéterminé?).")
                return None
            else:
                result_python_bool = bool(result_java_boolean)
                self._logger.info(f" -> Résultat requête Python: {result_python_bool}")
                return result_python_bool
        except jpype.JException as e_java:
            error_msg = f"Erreur Java exécution requête: {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Exécution Requête Tweety: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python exécution requête: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Exécution Requête: {e}") from e

    # --- Méthode Façade (ancienne, conservée pour compatibilité) ---
    @kernel_function(
        description="Exécute une requête en Logique Propositionnelle (syntaxe Tweety: !,||,=>,<=>,^^) sur un Belief Set fourni. Retourne le résultat (ACCEPTED, REJECTED, Unknown, ou FUNC_ERROR). NOTE: Préférez execute_and_log_pl_query qui enregistre automatiquement le résultat.",
        name="execute_pl_query"
    )
    def execute_pl_query(self, belief_set_content: str, query_string: str) -> str:
        self._logger.info(f"Appel execute_pl_query: Query='{query_string}' sur BS ('{belief_set_content[:60]}...')")
        if not self._jvm_ok:
            self._logger.error("execute_pl_query: JVM non prête.")
            return "FUNC_ERROR: JVM non prête ou composants Tweety non chargés."
        try:
            belief_set_obj = self._internal_parse_belief_set(belief_set_content)
            if belief_set_obj is None: return "FUNC_ERROR: Échec parsing Belief Set. Vérifiez syntaxe."
            formula_obj = self._internal_parse_formula(query_string)
            if formula_obj is None: return f"FUNC_ERROR: Échec parsing requête '{query_string}'. Vérifiez syntaxe."
            result_bool: Optional[bool] = self._internal_execute_query(belief_set_obj, formula_obj)
            if result_bool is None:
                result_str = f"Tweety Result: Unknown for query '{query_string}'."
                self._logger.warning(f"Requête '{query_string}' -> indéterminé (None).")
            else:
                result_label = "ACCEPTED (True)" if result_bool else "REJECTED (False)"
                result_str = f"Tweety Result: Query '{query_string}' is {result_label}."
                self._logger.info(f" -> Résultat formaté requête '{query_string}': {result_label}")
            return result_str
        except RuntimeError as e_runtime:
             self._logger.error(f"Erreur exécution (parsing/query): {e_runtime}")
             return f"FUNC_ERROR: {e_runtime}"
        except Exception as e_py:
            error_msg = f"Erreur Python inattendue dans execute_pl_query: {e_py}"
            self._logger.error(error_msg, exc_info=True)
            return f"FUNC_ERROR: {error_msg}"

    # --- Nouvelle Méthode Fusionnée (V12 - Fix callback params) ---
    @kernel_function(
        description="Exécute une requête PL ET enregistre automatiquement le résultat dans l'état. RECOMMANDÉE pour toutes les requêtes PL car elle garantit le logging pour la cross-validation.",
        name="execute_and_log_pl_query"
    )
    def execute_and_log_pl_query(self, belief_set_id: str, belief_set_content: str, query_string: str) -> str:
        """
        Exécute une requête PL et enregistre automatiquement le résultat.
        
        Args:
            belief_set_id: ID du belief set (ex: "propositional_bs_1")
            belief_set_content: Contenu du belief set en syntaxe Tweety
            query_string: Requête PL à exécuter
            
        Returns:
            Résultat formaté incluant confirmation du logging
        """
        self._logger.info(f"Appel execute_and_log_pl_query: Query='{query_string}' sur BS '{belief_set_id}'")
        
        # 1. Exécuter la requête
        result_str = self.execute_pl_query(belief_set_content, query_string)
        
        # 2. Parser le résultat pour extraire le statut
        if "ACCEPTED (True)" in result_str:
            status = "ACCEPTED"
        elif "REJECTED (False)" in result_str:
            status = "REJECTED"
        elif "Unknown" in result_str:
            status = "UNKNOWN"
        elif "FUNC_ERROR" in result_str:
            status = "ERROR"
        else:
            status = "UNKNOWN"
        
        # 3. Créer l'entrée de log
        log_entry = {
            "belief_set_id": belief_set_id,
            "query": query_string,
            "result": status,
            "raw_result": result_str,
            "timestamp": datetime.now().isoformat()
        }
        
        # 4. Logger en interne (toujours)
        self._query_log.append(log_entry)
        self._logger.info(f"Requête loggée en interne: {query_string} -> {status}")
        
        # 5. Appeler le callback externe si disponible (StateManager.log_query_result)
        # V12 FIX: Use correct parameter names matching StateManagerPlugin.log_query_result(belief_set_id, query, raw_result)
        callback_status = "non configuré"
        if self._log_callback is not None:
            try:
                # StateManagerPlugin.log_query_result expects: (belief_set_id: str, query: str, raw_result: str)
                callback_result = self._log_callback(
                    belief_set_id=belief_set_id,
                    query=query_string,  # Fixed: was 'query_string', now 'query' to match StateManagerPlugin
                    raw_result=result_str  # Fixed: was 'result' + 'details', now just 'raw_result'
                )
                callback_status = "OK"
                self._logger.info(f"Callback logging externe appelé avec succès.")
            except Exception as e:
                callback_status = f"ERREUR: {e}"
                self._logger.error(f"Erreur appel callback logging: {e}")
        
        # 6. Retourner résultat enrichi
        return f"{result_str}\n[LOGGED: BS={belief_set_id}, Status={status}, Callback={callback_status}]"


logger.info("Classe PropositionalLogicPlugin (V12 avec callback fix) définie.")

### Interpretation du code : PropositionalLogicPlugin (V12)

#### 1. Initialisation et verification JVM

```python
def __init__(self, log_callback: Optional[Callable] = None):
    self._logger = plugin_logger
    self._jvm_ok = False
    self._log_callback = log_callback
    self._query_log = []  # Log interne pour cross-validation
    self._initialize_jvm_components()
```

**Points cles:**
- Verifie que la JVM est demarree via la variable globale `jvm_ready`
- Charge les classes Java necessaires: `PlParser`, `SatReasoner`, `PlFormula`
- Configure le callback pour le logging automatique

**Gestion d'erreurs:**
- Si la JVM n'est pas prete, `_jvm_ok = False` et toutes les fonctions natives retourneront `FUNC_ERROR`
- Les erreurs de chargement des classes Java sont loggees en `CRITICAL`

#### 2. Parsing de formules et belief sets

Les methodes `_internal_parse_*` utilisent le `PlParser` de Tweety pour convertir du texte en objets Java.

**Syntaxe BNF Tweety (rappel):**

| Operateur | Signification | Exemple |
|-----------|---------------|---------|
| `!` | Negation | `!p` (non p) |
| `&&` | Conjonction | `p && q` (p et q) |
| `\|\|` | Disjonction | `p \|\| q` (p ou q) |
| `=>` | Implication | `p => q` (si p alors q) |
| `<=>` | Bi-implication | `p <=> q` (p ssi q) |
| `^^` | Ou exclusif | `p ^^ q` (xor) |

**Attention:** L'operateur `>>` existe dans la BNF mais est redondant avec `=>` et peut causer des confusions. Les prompts V10 demandent explicitement de l'eviter.

#### 3. Execution de requetes avec logging

La methode `execute_and_log_pl_query` est la **fonction recommandee** car elle combine:

1. **Execution** - Appel du `SatReasoner.query(belief_set, formula)`
2. **Parsing du resultat** - Conversion `Boolean` Java → `bool` Python → statut texte
3. **Logging interne** - Ajout dans `_query_log` avec timestamp
4. **Callback externe** - Appel automatique de `StateManager.log_query_result()` si configure

**Statuts possibles:**
- `ACCEPTED` - La requete est deduite du belief set
- `REJECTED` - La requete est contredite par le belief set
- `UNKNOWN` - Resultat indetermine (rare avec SatReasoner)
- `ERROR` - Erreur de parsing ou d'execution

#### 4. Fix V12 : Parametres du callback

La version V12 corrige un bug critique ou les parametres du callback ne correspondaient pas a la signature de `StateManager.log_query_result()`:

```python
# AVANT V12 (bug):
callback_result = self._log_callback(
    belief_set_id=belief_set_id,
    query_string=query_string,  # Mauvais nom de parametre
    result=status,
    details=result_str
)

# V12 (correct):
callback_result = self._log_callback(
    belief_set_id=belief_set_id,
    query=query_string,  # Nom correct
    raw_result=result_str  # Resultat complet
)
```

> **Impact pedagogique** : Ce type de bug est courant lors de l'integration de composants. Toujours verifier les signatures de fonctions lors de callbacks!

---

### 📜 Prompts Sémantiques et ⚙️ Fonction Setup (PL)

## Partie 2 : Prompts Semantiques - Pilotage LLM pour la Logique Formelle

### Objectif : Traduire langage naturel ↔ logique formelle

L'agent PL doit effectuer trois taches complexes necessitant un LLM:

1. **Traduction texte → PL** - Convertir des arguments en langage naturel en formules logiques
2. **Generation de requetes** - Identifier les questions pertinentes a poser au belief set
3. **Interpretation resultat → texte** - Expliquer les resultats formels en langage comprehensible

Chacune de ces taches est pilotee par un **prompt semantique** optimise.

### Evolution des prompts : V8 → V10

| Version | Changement principal |
|---------|---------------------|
| V8 | Operateurs de base (`!`, `\|\|`, `=>`, `<=>`) |
| V9 | Ajout de l'operateur `^^` (xor) |
| V10 | **Ajout de `&&` (conjonction)** - Operateur manquant dans V8/V9! |

**Pourquoi V10 est crucial:**

La conjonction `&&` est fondamentale en logique propositionnelle. Les versions V8/V9 etaient **incompletes** car elles ne permettaient pas d'exprimer facilement "p ET q" (fallait utiliser `!((!p) || (!q))` via De Morgan, peu lisible).

### Syntaxe Tweety complete (reference BNF)

D'apres le code source officiel de `PlParser.java`:

```bnf
FORMULA ::== PROPOSITION 
          | "(" FORMULA ")" 
          | FORMULA ">>" FORMULA
          | FORMULA "||" FORMULA 
          | FORMULA "&&" FORMULA 
          | FORMULA "=>" FORMULA 
          | FORMULA "<=>" FORMULA 
          | FORMULA "^^" FORMULA 
          | "!" FORMULA 
          | "+" 
          | "-"
```

**Operateurs speciaux:**
- `+` : Tautologie (toujours vrai)
- `-` : Contradiction (toujours faux)
- `>>` : Implication materielle (redondant avec `=>`, **a eviter**)

> **Note pedagogique** : Les prompts V10 demandent explicitement d'eviter `>>` pour eviter les confusions. L'operateur `=>` est plus standard en logique formelle.

---

In [ ]:
# %% CELLULE [6.2] - Prompts Semantiques et Fonction Setup (PL) - V10
# (Remplace une partie de l'ancienne cellule 7bbc31e5 et corrige l'indentation)
# V10 - Fix: Ajoute && (conjonction) aux operateurs autorises selon BNF Tweety officielle
# Reference: https://github.com/TweetyProjectTeam/TweetyProject/blob/main/org-tweetyproject-logics-pl/src/main/java/org/tweetyproject/logics/pl/parser/PlParser.java

import semantic_kernel as sk
from semantic_kernel.functions import kernel_function
import logging

# Tenter d'importer les classes de configuration de prompt (SK 1.0+)
try:
    from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
    SK_HAS_INPUT_VARIABLE = True
    logger_setup = logging.getLogger("Orchestration.AgentPL.Setup")
    logger_setup.debug("SK 1.0+ detecte avec InputVariable/PromptTemplateConfig")
except ImportError:
    SK_HAS_INPUT_VARIABLE = False

# S'assurer que les dependances sont la
if 'PropositionalLogicPlugin' not in globals(): raise NameError("Classe PropositionalLogicPlugin non definie.")

logger = logging.getLogger("Orchestration.AgentPL.Setup")

# --- Fonctions Semantiques PLAnalyzer (Prompts V10 - Tous operateurs Tweety) ---
# BNF Tweety officielle (PlParser.java):
# FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
#              FORMULA "||" FORMULA | FORMULA "&&" FORMULA | FORMULA "=>" FORMULA | 
#              FORMULA "<=>" FORMULA | FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
#
# Operateurs supportes:
#   !     negation
#   &&    conjonction (et)
#   ||    disjonction (ou)
#   =>    implication (si...alors)
#   <=>   bi-implication (si et seulement si)
#   ^^    ou exclusif (xor)
#   >>    implication materielle (eviter, redondant avec =>)
#   +     tautologie
#   -     contradiction

# V10: Prompts avec tous les operateurs Tweety valides (inclut &&)
prompt_text_to_pl_v10 = """
[Instructions]
Transformez le texte fourni en un belief set en Logique Propositionnelle (PL).
Retournez les formules une par ligne, separees par des sauts de ligne.

Operateurs Tweety disponibles:
- ! pour la negation (ex: !p signifie "non p")
- && pour la conjonction (ex: p && q signifie "p et q")
- || pour la disjonction (ex: p || q signifie "p ou q")
- => pour l'implication (ex: p => q signifie "si p alors q")
- <=> pour la bi-implication (ex: p <=> q signifie "p si et seulement si q")
- ^^ pour le ou exclusif (xor)

Regles:
- Evitez l'operateur >> (utilisez => a la place)
- Utilisez des noms de propositions courts et significatifs en minuscules avec underscores
- Exemples: renewable_essential, high_cost, creates_jobs, climate_action
- Preferez les formules simples et lisibles

[Texte a Analyser]
{{$input}}

[Belief Set PL - une formule par ligne]
"""

prompt_gen_pl_queries_v10 = """
[Instructions]
Etant donne un texte original et un belief set PL, generez 2-3 requetes PL pertinentes.
Les requetes doivent verifier la coherence ou deduire des informations du belief set.

Operateurs Tweety disponibles:
- ! pour la negation
- && pour la conjonction (et)
- || pour la disjonction (ou)
- => pour l'implication
- <=> pour la bi-implication
- ^^ pour le ou exclusif

Regles:
- Evitez l'operateur >> (utilisez => a la place)
- Assurez-vous que les propositions utilisees existent dans le belief set
- Retournez les requetes une par ligne

[Texte Original]
{{$input}}

[Belief Set PL]
{{$belief_set}}

[Requetes PL Generees - une par ligne]
"""

prompt_interpret_pl_v10 = """
[Instructions]
Interpretez en langage naturel clair le resultat Tweety pour les requetes PL.
Le resultat pour chaque requete indique si elle est ACCEPTED (True), REJECTED (False), ou Unknown.

Pour chaque requete:
1. Rappelez la requete
2. Indiquez si elle est Acceptee, Rejetee ou Inconnue
3. Expliquez pourquoi en vous referant aux formules du belief set
4. Reliez l'interpretation au sens du texte original

[Texte Original]
{{$input}}

[Belief Set PL]
{{$belief_set}}

[Requetes Testees - une par ligne]
{{$queries}}

[Resultats Tweety - un resultat par ligne]
{{$tweety_result}}

[Interpretation en Langage Naturel]
"""

# Variables de compatibilite
prompt_text_to_pl_v8 = prompt_text_to_pl_v10
prompt_text_to_pl_v9 = prompt_text_to_pl_v10
prompt_gen_pl_queries_v8 = prompt_gen_pl_queries_v10
prompt_gen_pl_queries_v9 = prompt_gen_pl_queries_v10
prompt_interpret_pl_v8 = prompt_interpret_pl_v10
prompt_interpret_pl_v9 = prompt_interpret_pl_v10

logger.debug("Prompts semantiques PL (V10 - Tous operateurs Tweety) definis.")

# --- Fonction pour configurer le Kernel specifique a l'agent PL (V12 - allow_dangerously_set_content) ---
def setup_pl_kernel(kernel: sk.Kernel, llm_service, state_manager_plugin=None):
    """
    Configure le kernel pour le PropositionalLogicAgent.
    Ajoute une instance du PropositionalLogicPlugin et les fonctions semantiques.
    
    Args:
        kernel: Kernel Semantic Kernel a configurer
        llm_service: Service LLM pour les fonctions semantiques
        state_manager_plugin: Instance optionnelle du StateManagerPlugin pour configurer le callback logging
    """
    plugin_name = "PLAnalyzer"
    logger.info(f"Configuration Kernel pour {plugin_name} (V12 - allow_dangerously_set_content)...")

    # Configurer le callback de logging si state_manager_plugin est fourni
    log_callback = None
    if state_manager_plugin is not None:
        if hasattr(state_manager_plugin, 'log_query_result'):
            log_callback = state_manager_plugin.log_query_result
            logger.info("Callback log_query_result configure depuis StateManagerPlugin.")
        else:
            logger.warning("StateManagerPlugin fourni mais sans methode log_query_result.")

    # Instanciation du plugin avec le callback
    pl_plugin_instance = PropositionalLogicPlugin(log_callback=log_callback)

    # Ajout du plugin au kernel passe en argument
    if plugin_name in kernel.plugins: logger.warning(f"Plugin '{plugin_name}' deja present. Remplacement.")
    kernel.add_plugin(pl_plugin_instance, plugin_name=plugin_name)
    logger.debug(f"Instance du plugin '{plugin_name}' ajoutee/mise a jour.")

    # Configuration des settings LLM
    default_settings = None
    if llm_service:
        try: default_settings = kernel.get_prompt_execution_settings_from_service_id(llm_service.service_id); logger.debug(f"Settings LLM recuperes pour {plugin_name}.")
        except Exception as e: logger.warning(f"Impossible de recuperer settings LLM pour {plugin_name}: {e}")

    # Configuration avec allow_dangerously_set_content pour SK 1.0+
    semantic_functions_config = [
        ("semantic_TextToPLBeliefSet", prompt_text_to_pl_v10, "Traduit texte en Belief Set PL (syntaxe Tweety)."),
        ("semantic_GeneratePLQueries", prompt_gen_pl_queries_v10, "Genere requetes PL pertinentes (syntaxe Tweety)."),
        ("semantic_InterpretPLResult", prompt_interpret_pl_v10, "Interprete resultat requete PL Tweety formate.")
    ]
    
    for func_name, prompt, description in semantic_functions_config:
        try:
            if SK_HAS_INPUT_VARIABLE:
                # SK 1.0+ avec PromptTemplateConfig et allow_dangerously_set_content
                # Determiner les variables d'entree selon la fonction
                if func_name == "semantic_TextToPLBeliefSet":
                    input_vars = [
                        InputVariable(name="input", description="Texte a traduire en PL", 
                                     is_required=True, allow_dangerously_set_content=True)
                    ]
                elif func_name == "semantic_GeneratePLQueries":
                    input_vars = [
                        InputVariable(name="input", description="Texte original", 
                                     is_required=True, allow_dangerously_set_content=True),
                        InputVariable(name="belief_set", description="Belief Set PL", 
                                     is_required=True, allow_dangerously_set_content=True)
                    ]
                else:  # semantic_InterpretPLResult
                    input_vars = [
                        InputVariable(name="input", description="Texte original", 
                                     is_required=True, allow_dangerously_set_content=True),
                        InputVariable(name="belief_set", description="Belief Set PL", 
                                     is_required=True, allow_dangerously_set_content=True),
                        InputVariable(name="queries", description="Requetes testees", 
                                     is_required=True, allow_dangerously_set_content=True),
                        InputVariable(name="tweety_result", description="Resultats Tweety", 
                                     is_required=True, allow_dangerously_set_content=True)
                    ]
                
                prompt_config = PromptTemplateConfig(
                    template=prompt,
                    name=func_name,
                    description=description,
                    input_variables=input_vars
                )
                
                kernel.add_function(
                    prompt_template_config=prompt_config,
                    plugin_name=plugin_name,
                    function_name=func_name,
                    prompt_execution_settings=default_settings
                )
                logger.debug(f"Fonction {plugin_name}.{func_name} ajoutee (SK 1.0+ avec config).")
            else:
                # Fallback pour anciennes versions SK
                kernel.add_function(
                    prompt=prompt, 
                    plugin_name=plugin_name, 
                    function_name=func_name, 
                    description=description, 
                    prompt_execution_settings=default_settings
                )
                logger.debug(f"Fonction {plugin_name}.{func_name} ajoutee (mode legacy).")
                
        except ValueError as ve: 
            logger.warning(f"Probleme ajout/MaJ {plugin_name}.{func_name}: {ve}")
        except Exception as e:
            logger.error(f"Erreur ajout {plugin_name}.{func_name}: {e}")

    # Verification des fonctions natives (facade + nouvelle fusionnee)
    native_functions = ["execute_pl_query", "execute_and_log_pl_query"]
    if plugin_name in kernel.plugins:
        for native_func in native_functions:
            if native_func not in kernel.plugins[plugin_name]:
                logger.error(f"ERREUR: Fonction native {plugin_name}.{native_func} non enregistree!")
            else:
                logger.debug(f"Fonction native {plugin_name}.{native_func} trouvee.")
    else:
        logger.error(f"ERREUR CRITIQUE: Plugin {plugin_name} non trouve apres ajout!")

    logger.info(f"Kernel {plugin_name} configure (V12).")
    
    # Retourner l'instance du plugin pour permettre l'acces au query_log
    return pl_plugin_instance

# Fin de la definition de la fonction setup_pl_kernel

### Interpretation des prompts semantiques

#### Prompt 1 : `prompt_text_to_pl_v10` - Traduction texte → PL

**Entree:** Texte en langage naturel contenant des arguments.

**Sortie attendue:** Liste de formules PL, une par ligne.

**Exemple:**

```
Entree:
"Les energies renouvelables sont essentielles. Elles coutent cher mais creent des emplois."

Sortie:
renewable_essential
high_cost
creates_jobs
renewable_essential => (high_cost && creates_jobs)
```

**Points pedagogiques:**
- Le LLM doit identifier les **propositions atomiques** (faits de base)
- Puis les **relations logiques** entre ces propositions
- Les noms de propositions doivent etre courts et descriptifs (snake_case recommande)

#### Prompt 2 : `prompt_gen_pl_queries_v10` - Generation de requetes

**Entrees:** Texte original + Belief set PL.

**Sortie attendue:** 2-3 requetes PL pertinentes.

**Exemple:**

```
Belief Set:
renewable_essential
high_cost
creates_jobs
renewable_essential => (high_cost && creates_jobs)

Requetes generees:
renewable_essential
high_cost && creates_jobs
!renewable_essential || creates_jobs
```

**Strategie du prompt:**
- Les requetes doivent **utiliser uniquement** les propositions presentes dans le belief set
- Verifier la **coherence** (ex: tester si une implication est respectee)
- Deduire des **consequences** (ex: si A => B et A, alors B?)

#### Prompt 3 : `prompt_interpret_pl_v10` - Interpretation des resultats

**Entrees:** Texte original + Belief set + Requetes + Resultats Tweety.

**Sortie attendue:** Explication en langage naturel.

**Exemple:**

```
Resultat Tweety:
Query 'renewable_essential' is ACCEPTED (True).
Query 'high_cost && creates_jobs' is ACCEPTED (True).

Interpretation:
La requete 'renewable_essential' est acceptee car elle figure directement dans le belief set.
La requete 'high_cost && creates_jobs' est acceptee car les deux propositions sont presentes
et leur conjonction est deduite de l'implication 'renewable_essential => (high_cost && creates_jobs)'
combinee avec le fait 'renewable_essential'.
```

**Role pedagogique:**
- Rendre les resultats formels **comprehensibles** pour un utilisateur non-expert
- Relier les deductions logiques au **sens original** des arguments

---

### Fonction `setup_pl_kernel()` - Configuration du Semantic Kernel

Cette fonction effectue plusieurs operations critiques:

1. **Instanciation du plugin** avec callback de logging
2. **Ajout au kernel** sous le nom `PLAnalyzer`
3. **Configuration des prompts** avec `allow_dangerously_set_content=True`
4. **Verification** de la presence des fonctions natives

#### Pourquoi `allow_dangerously_set_content=True` ?

Le Semantic Kernel 1.0+ introduit une protection contre l'injection de contenu malveillant dans les prompts. Cependant, notre cas d'usage **legitime** necessite d'injecter:
- Du texte utilisateur brut (arguments a analyser)
- Des formules PL generees dynamiquement
- Des resultats Tweety bruts

**Sans cette option**, SK refuserait d'executer les prompts avec erreur de securite.

> **Attention securite** : Cette option doit etre utilisee uniquement dans un environnement controle. En production, il faudrait ajouter une validation/sanitization du contenu avant injection.

#### Variables d'entree par fonction

| Fonction | Variables requises | Description |
|----------|-------------------|-------------|
| `semantic_TextToPLBeliefSet` | `input` | Texte a traduire |
| `semantic_GeneratePLQueries` | `input`, `belief_set` | Texte + BS existant |
| `semantic_InterpretPLResult` | `input`, `belief_set`, `queries`, `tweety_result` | Tout le contexte |

**Note SK 1.0+** : Chaque variable est definie avec `InputVariable()` qui specifie:
- `name` : Nom de la variable dans le prompt (ex: `{{$input}}`)
- `description` : Documentation pour le developpeur
- `is_required` : Si `True`, SK verifie la presence avant execution
- `allow_dangerously_set_content` : Autorise l'injection de contenu non valide

#### Retour de la fonction

La fonction `setup_pl_kernel()` retourne l'instance du plugin (`pl_plugin_instance`). Ceci permet d'acceder au log des requetes executees:

```python
pl_plugin = setup_pl_kernel(kernel, llm_service, state_manager_plugin)
# Plus tard...
query_log = pl_plugin.get_query_log()  # Recuperer toutes les requetes executees
```

**Utilite pedagogique** : Permet d'auditer toutes les requetes executees par l'agent pour analyse ou debugging.

---

### 📜 Instructions Système : PL_AGENT_INSTRUCTIONS

## Partie 3 : Instructions Systeme - Regles Strictes de Comportement

### Problematique : Controle d'un agent autonome

Les agents LLM ont tendance a :
- **Ecrire du texte libre** plutot que d'appeler des fonctions
- **Depasser leur role** en creant des taches ou delegant a d'autres agents
- **Oublier d'enregistrer leurs resultats** dans l'etat partage

Les instructions systeme definissent des **regles strictes** pour eviter ces comportements problematiques.

### Evolution V12 → V16 : Interdictions renforcees

| Version | Changements |
|---------|-------------|
| V12 | Ajout callback logging automatique |
| V15 | Interdiction explicite de `add_analysis_task` |
| V16 | **Interdiction de `designate_next_agent`** - Seul le PM delegue |

**Pourquoi ces interdictions ?**

Sans ces regles, un agent PL pourrait :
1. Creer sa propre tache → **Duplication de travail**
2. Deleguer a un autre agent → **Court-circuitage du PM**
3. Ecrire du texte au lieu d'appeler `add_answer` → **Perte de resultats**

### Principe : Separation stricte des responsabilites

| Agent | Autorisations | Interdictions |
|-------|---------------|---------------|
| **PropositionalLogicAgent** | Traduire en PL, Executer requetes, Enregistrer resultats | Creer taches, Deleguer, Conclure globalement |
| **ProjectManagerAgent** | Creer taches, Deleguer, Conclure | Executer analyses techniques |

Cette architecture suit le pattern **Command/Executor** :
- Le **PM** commande (cree des taches)
- Les **agents specialises** executent (et reportent via `add_answer`)

---

In [ ]:
# %% CELLULE [6.3] - Instructions Systeme (PL)
# V16 - INTERDIT add_analysis_task et designate_next_agent

import logging

logger = logging.getLogger("Orchestration.AgentPL.Instructions")

# --- Instructions Systeme PL Agent (V16 - INTERDIT add_analysis_task) ---
PL_AGENT_INSTRUCTIONS_V16 = """
Votre Role: Specialiste en logique propositionnelle (Tweety).

**REGLES CRITIQUES D'EXECUTION:**

1. **NE JAMAIS ECRIRE DE TEXTE LIBRE** - Ecrire du texte termine votre tour SANS appeler add_answer!
2. **UNIQUEMENT DES APPELS DE FONCTION** - Tout votre travail passe par StateManager et PLAnalyzer.
3. **add_answer EST OBLIGATOIRE** - C'est TOUJOURS votre DERNIER appel de fonction.
4. **INTERDIT: set_final_conclusion** - Reserve au ProjectManagerAgent.
5. **INTERDIT: add_analysis_task** - SEUL le ProjectManagerAgent peut creer des taches!
6. **INTERDIT: designate_next_agent** - SEUL le ProjectManagerAgent gere la delegation!

**Syntaxe Tweety (BNF officielle):**
- ! negation
- && conjonction (et)
- || disjonction (ou)
- => implication
- <=> bi-implication
- ^^ ou exclusif
- Evitez >> (utilisez => a la place)

**Fonctions AUTORISEES:**
- `StateManager.get_current_state_snapshot(summarize)`: Lire l'etat.
- `StateManager.add_belief_set(logic_type, content)`: Enregistrer un belief set.
- `StateManager.add_answer(task_id, author_agent, answer_text, source_ids)`: **OBLIGATOIRE - FIN DE TACHE**
- `PLAnalyzer.semantic_TextToPLBeliefSet(input)`: Traduire en PL.
- `PLAnalyzer.semantic_GeneratePLQueries(input, belief_set)`: Generer requetes.
- `PLAnalyzer.execute_and_log_pl_query(belief_set_id, belief_set_content, query_string)`: **RECOMMANDE** - Execute ET logue.
- `PLAnalyzer.semantic_InterpretPLResult(...)`: Interpreter resultats.

**Fonctions INTERDITES (reservees au PM):**
- `StateManager.add_analysis_task` - INTERDIT!
- `StateManager.designate_next_agent` - INTERDIT!
- `StateManager.set_final_conclusion` - INTERDIT!

**WORKFLOW OBLIGATOIRE - TRADUCTION PL:**

```
ETAPE 1: Lire l'etat
  -> StateManager.get_current_state_snapshot(summarize=False)

ETAPE 2: Traduire en PL
  -> PLAnalyzer.semantic_TextToPLBeliefSet(input="[texte arguments]")

ETAPE 3: Enregistrer le belief set
  -> StateManager.add_belief_set(logic_type="Propositional", content="[formules]")

ETAPE 4: OBLIGATOIRE - Signaler fin
  -> StateManager.add_answer(
       task_id="task_X",
       author_agent="PropositionalLogicAgent",
       answer_text="Belief set cree avec N formules.",
       source_ids=["propositional_bs_1"]
     )

ETAPE 5: STOP - NE RIEN FAIRE DE PLUS (pas de texte, pas de nouvelle tache!)
```

**WORKFLOW OBLIGATOIRE - REQUETES PL:**

```
ETAPE 1: Lire l'etat (recuperer belief_set_id et content)
  -> StateManager.get_current_state_snapshot(summarize=False)

ETAPE 2: Generer requetes
  -> PLAnalyzer.semantic_GeneratePLQueries(input, belief_set)

ETAPE 3: Executer 2-3 requetes MAX (avec logging auto)
  -> PLAnalyzer.execute_and_log_pl_query(bs_id, bs_content, query1)
  -> PLAnalyzer.execute_and_log_pl_query(bs_id, bs_content, query2)

ETAPE 4: OBLIGATOIRE - Signaler fin
  -> StateManager.add_answer(
       task_id="task_X",
       author_agent="PropositionalLogicAgent",
       answer_text="N requetes executees.",
       source_ids=[]
     )

ETAPE 5: STOP - NE RIEN FAIRE DE PLUS (pas de texte, pas de nouvelle tache!)
```

**ERREURS A EVITER:**
- Ecrire "Voici les resultats..." -> TERMINE LE TOUR SANS add_answer!
- Appeler add_analysis_task -> INTERDIT! Seul le PM cree des taches!
- Executer trop de requetes (>3) -> Risque de timeout!
- Oublier add_answer -> PM bloque en attente indefiniment!

**RAPPEL FINAL:**
Votre tour se termine des que vous ecrivez du texte.
Appelez add_answer AVANT tout texte, ou mieux: N'ECRIVEZ PAS DE TEXTE DU TOUT.
NE CREEZ JAMAIS DE TACHE - c'est le travail du ProjectManagerAgent!
""" 

PL_AGENT_INSTRUCTIONS_V15 = PL_AGENT_INSTRUCTIONS_V16
PL_AGENT_INSTRUCTIONS = PL_AGENT_INSTRUCTIONS_V16

logger.info("Instructions Systeme PL_AGENT_INSTRUCTIONS (V16 - INTERDIT add_analysis_task) definies.")

### Analyse des instructions systeme (V16)

#### Section 1 : REGLES CRITIQUES D'EXECUTION

```
1. NE JAMAIS ECRIRE DE TEXTE LIBRE - Ecrire du texte termine votre tour SANS appeler add_answer!
2. UNIQUEMENT DES APPELS DE FONCTION - Tout votre travail passe par StateManager et PLAnalyzer.
3. add_answer EST OBLIGATOIRE - C'est TOUJOURS votre DERNIER appel de fonction.
```

**Explication technique:**

Le Semantic Kernel fonctionne en **tours de conversation**. Quand un agent ecrit du texte (sans balises de fonction), le kernel considere que le tour est termine et rend la main.

**Probleme:** Si l'agent ecrit du texte AVANT d'appeler `add_answer`, ses resultats ne sont jamais enregistres dans l'etat partage.

**Solution:** Les instructions martellent ce point pour forcer l'agent a **TOUJOURS** appeler `add_answer` en dernier (ou mieux, ne jamais ecrire de texte).

#### Section 2 : Syntaxe Tweety

Rappel des operateurs avec la recommandation explicite d'eviter `>>`.

> **Pedagogie** : Meme si la BNF autorise `>>`, l'utiliser peut creer des confusions car il existe deux notations pour l'implication (`=>` et `>>`). Les instructions standardisent sur `=>`.

#### Section 3 : Fonctions AUTORISEES vs INTERDITES

**Autorisees (agent PL):**
- `StateManager.get_current_state_snapshot()` - Lecture seule de l'etat
- `StateManager.add_belief_set()` - Enregistrer un BS
- `StateManager.add_answer()` - **OBLIGATOIRE** - Fin de tache
- `PLAnalyzer.semantic_*()` - Fonctions LLM
- `PLAnalyzer.execute_and_log_pl_query()` - **RECOMMANDE** - Execute + logue

**Interdites:**
- `StateManager.add_analysis_task` - Reserve au PM
- `StateManager.designate_next_agent` - Reserve au PM
- `StateManager.set_final_conclusion` - Reserve au PM

**Impact pedagogique:**

Cette liste explicite agit comme une **whitelist**. L'agent sait exactement ce qu'il peut faire. Toute autre action est interdite.

#### Section 4 : WORKFLOW OBLIGATOIRE - TRADUCTION PL

Les instructions fournissent un **script detaille** en 5 etapes:

```
ETAPE 1: Lire l'etat
ETAPE 2: Traduire en PL
ETAPE 3: Enregistrer le belief set
ETAPE 4: Signaler fin avec add_answer
ETAPE 5: STOP - Ne rien faire de plus
```

**Pourquoi un workflow aussi prescriptif ?**

Sans cela, l'agent LLM pourrait :
- Sauter des etapes (ex: traduire sans enregistrer)
- Ajouter des etapes non necessaires (ex: interpreter immediatement)
- Oublier de signaler la fin

Le workflow garantit la **reproductibilite** et la **completude**.

#### Section 5 : WORKFLOW OBLIGATOIRE - REQUETES PL

Workflow similaire mais avec une limite explicite : **2-3 requetes MAX**.

**Pourquoi limiter le nombre de requetes ?**

1. **Performance** - Chaque requete necessite un appel Java/JPype (~50-200ms)
2. **Timeout** - Les agents ont un timeout global (ex: 60s)
3. **Pertinence** - Plus de requetes ≠ meilleure analyse. Mieux vaut 2-3 requetes ciblees.

**Note V16** : Les instructions mentionnent explicitement `execute_and_log_pl_query` comme fonction **RECOMMANDEE** (en majuscules) pour garantir le logging automatique.

#### Section 6 : ERREURS A EVITER

Liste des erreurs courantes observees lors des tests:

| Erreur | Consequence | Solution |
|--------|-------------|----------|
| Ecrire "Voici les resultats..." | Tour termine sans `add_answer` | NE PAS ECRIRE DE TEXTE |
| Appeler `add_analysis_task` | Duplication de taches | Rappel: INTERDIT |
| Executer >3 requetes | Timeout | Limiter a 2-3 requetes |
| Oublier `add_answer` | PM bloque indefiniment | Toujours appeler en dernier |

**Impact pedagogique:**

Cette section anticipe les erreurs et les previent. C'est le fruit de **tests iteratifs** : chaque erreur observee a ete ajoutee aux instructions pour eviter sa recurrence.

#### RAPPEL FINAL

```
Votre tour se termine des que vous ecrivez du texte.
Appelez add_answer AVANT tout texte, ou mieux: N'ECRIVEZ PAS DE TEXTE DU TOUT.
NE CREEZ JAMAIS DE TACHE - c'est le travail du ProjectManagerAgent!
```

**Philosophie de conception:**

Ces instructions refletent une approche **defensive** : mieux vaut etre trop prescriptif que laisser l'agent improviser. Les LLMs sont puissants mais impredictibles. Les contraintes strictes garantissent la fiabilite du systeme multi-agents.

---

## 6.1. Test du Plugin PropositionalLogicPlugin (Tweety) - Commenté

Cette cellule, **commentée par défaut**, contient du code pour tester isolément le `PropositionalLogicPlugin`, en particulier son interaction avec Tweety via JPype.

**Objectif du test (s'il était activé) :**
*   Vérifier que la JVM est prête.
*   Instancier le plugin.
*   Exécuter la fonction native `execute_pl_query` avec un belief set et des requêtes prédéfinis.
*   Analyser les résultats (ACCEPTED/REJECTED/Error).

**Prérequis INDISPENSABLES (si activé) :**
1.  La **JVM doit être démarrée correctement** (voir cellule 2). La variable `jvm_ready` doit être `True`.
2.  La cellule de définition du `PropositionalLogicPlugin` doit avoir été exécutée.

**Statut actuel :** Laissé commenté pour se concentrer sur le flux principal. Peut être utile pour déboguer l'interaction JPype/Tweety si l'agent PL rencontre des problèmes.

In [ ]:
# # %% Test du plugin PropositionalLogicPlugin (V6 - Contradiction Explicite)
# import logging
# import jpype # Importer pour vérifier

# test_pl_logger = logging.getLogger("Orchestration.Test.PLPlugin")
# test_pl_logger.setLevel(logging.INFO)
# if not test_pl_logger.handlers:
#     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); test_pl_logger.addHandler(handler)

# test_pl_logger.info("--- Début Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")

# # Vérifier prérequis
# jvm_ok_for_test = False
# if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
#     test_pl_logger.info("✅ Prérequis JVM OK.")
#     jvm_ok_for_test = True
# else:
#     test_pl_logger.error("❌ Prérequis JVM NON rempli. Le test natif échouera probablement.")
#     print("❌ ERREUR: JVM non prête. Exécutez la cellule de configuration JPype.")

# if 'PropositionalLogicPlugin' not in globals():
#      test_pl_logger.error("❌ Classe PropositionalLogicPlugin non trouvée.")
#      print("❌ ERREUR: Classe PropositionalLogicPlugin non définie.")
#      jvm_ok_for_test = False # Empêcher la suite

# if jvm_ok_for_test:
#     try:
#         # 1. Instanciation
#         test_pl_logger.info("1. Instanciation de PropositionalLogicPlugin...")
#         plugin_instance_pl_test = PropositionalLogicPlugin()
#         test_pl_logger.info("   Instance créée.")

#         # 2. Définition données de test (CORRIGÉ V6: Ajout 'imported_fossils' pour rendre la contradiction claire)
#         test_belief_set = """essential_law
# !high_cost || !false_debate
# false_debate || long_term_cost_inaction
# high_cost
# renewable_creates_jobs
# imported_fossils
# !imported_fossils || dependency
# # La ligne '!renewable_creates_jobs || !dependency' est enlevée car elle contredit directement les faits 5 et 6/7
# irresponsible_ignore
# !essential_law || !long_term_cost_inaction || !renewable_creates_jobs || !dependency || !irresponsible_ignore
# """
#         # Les requêtes restent les mêmes
#         test_query_1 = "essential_law" # Attendu: ACCEPTED
#         test_query_2 = "long_term_cost_inaction" # Attendu: ACCEPTED
#         test_query_3 = "cheap_law" # Attendu: REJECTED/Unknown
#         test_query_4 = "!irresponsible_ignore" # Attendu: ACCEPTED (car KB contradictoire)
#         test_query_5 = "non_existent" # Attendu: REJECTED/Unknown
#         test_query_6 = "essential_law || !essential_law" # Tautologie, Attendu: ACCEPTED
#         test_query_7 = "essential_law && !essential_law" # Contradiction, Attendu: REJECTED (si KB était cohérent), ACCEPTED (si KB contradictoire)

#         print("\n--- Données de Test (Contradiction Explicite) ---")
#         print(f"Belief Set:\n{test_belief_set}")
#         queries_to_test = [test_query_1, test_query_2, test_query_3, test_query_4, test_query_5, test_query_6, test_query_7]
#         for i, q in enumerate(queries_to_test): print(f"Requête {i+1}: {q}")


#         # 3. Exécution des requêtes
#         test_pl_logger.info("\n2. Test: native_ExecutePLQuery sur Belief Set Complet V6...")

#         print("\n--- Résultats Tests Requêtes ---")
#         all_queries_passed = True # Optimiste
#         results_summary = []

#         for i, query in enumerate(queries_to_test):
#             test_pl_logger.info(f"   Exécution requête {i+1}: '{query}'")
#             result = plugin_instance_pl_test.native_ExecutePLQuery(test_belief_set, query)
#             test_pl_logger.info(f"   Résultat brut requête {i+1}: {result}")
#             print(f"Résultat Requête {i+1} ('{query}'): {result}")
#             results_summary.append(result)

#             # Vérification (ATTENTION: si KB contradictoire, tout devrait être ACCEPTED sauf erreur)
#             is_error = "FUNC_ERROR:" in result
#             is_accepted = "ACCEPTED (True)" in result
#             is_rejected = "REJECTED (False)" in result or "Unknown" in result # Grouper REJECTED et Unknown

#             if is_error:
#                 test_pl_logger.error(f"   !!! Erreur détectée pour la requête {i+1} : {result} !!!")
#                 all_queries_passed = False
#             elif i == 2 or i == 4 : # cheap_law (Query 3), non_existent (Query 5) -> Devrait être ACCEPTED si KB contradictoire
#                  if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#                  else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # else: # Toutes les autres requêtes (1, 2, 4, 6, 7) devraient être ACCEPTED si KB contradictoire
#             #      if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#             #      else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # Simplification: on s'attend à ce que tout soit ACCEPTED
#             else:
#                 if not is_accepted:
#                     all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire, obtenu: {'REJECTED/Unknown' if is_rejected else result})")
#                 else:
#                     test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")


#         print("\n--- Vérification Globale Requêtes (Attente: Tout ACCEPTED car KB contradictoire) ---")
#         if all_queries_passed:
#             test_pl_logger.info("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED, comportement attendu pour un KB contradictoire.")
#             print("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED.")
#         else:
#             test_pl_logger.error("❌❌❌ Le comportement face à la contradiction est inattendu. Certaines requêtes n'ont pas retourné ACCEPTED. Vérifiez les logs et le comportement de Tweety SatReasoner.")
#             print("❌❌❌ Comportement inattendu face à la contradiction. Vérifiez les logs.")
#             print("Résultats obtenus :", results_summary)


#     except Exception as e:
#         test_pl_logger.critical(f"Erreur majeure lors du test du plugin PL: {e}", exc_info=True)
#         print(f"\n !!! ERREUR CRITIQUE PENDANT LE TEST PL : {e} !!!")
# else:
#     print("\n--- Test PL Plugin Sauté (Prérequis non remplis) ---")

# test_pl_logger.info("--- Fin Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")

## Conclusion et Recapitulatif

### Synthese des composants de l'agent PL

Nous avons defini trois composants essentiels pour le PropositionalLogicAgent:

| Composant | Fichier/Variable | Role |
|-----------|------------------|------|
| **Plugin natif** | `PropositionalLogicPlugin` (classe Python) | Interface JPype/Tweety, execution de requetes formelles |
| **Prompts semantiques** | `prompt_text_to_pl_v10`, etc. | Pilotage LLM pour traduction NL ↔ PL |
| **Instructions systeme** | `PL_AGENT_INSTRUCTIONS_V16` | Regles strictes de comportement |
| **Fonction setup** | `setup_pl_kernel()` | Configuration du Semantic Kernel |

### Points cles a retenir

1. **Integration Java/Python** - JPype permet d'utiliser des bibliotheques Java complexes (Tweety) depuis Python sans reimplementation.

2. **Logging automatique** - La methode `execute_and_log_pl_query` garantit la tracabilite de toutes les requetes pour la cross-validation.

3. **Prompts evolutifs** - Les versions V8 → V10 montrent l'importance de la completude de la syntaxe (ajout de `&&`).

4. **Instructions strictes** - Les regles V16 refletent un apprentissage iteratif : chaque interdiction correspond a une erreur observee.

5. **Separation des responsabilites** - Le pattern Command/Executor assure que seul le PM orchestre, les agents specialises executent.

### Utilisation dans le workflow global

Le PropositionalLogicAgent sera instancie dans le notebook d'orchestration (Notebook 3) et recevra des taches du ProjectManagerAgent:

```
ProjectManagerAgent
    |
    v (cree tache "translate_to_pl")
PropositionalLogicAgent
    |
    +-> semantic_TextToPLBeliefSet (LLM)
    +-> add_belief_set (StateManager)
    +-> add_answer (StateManager)
    v
Resultats enregistres dans l'etat partage
```

### Prochaines etapes

Le **Notebook 3 (Orchestration)** combinera:
- InformalAnalysisAgent (Notebook 1)
- PropositionalLogicAgent (Notebook 2)
- ProjectManagerAgent (a definir)
- StateManager (etat partage)

Pour une analyse d'arguments complete et automatisee.

---

**Note pedagogique finale:**

Ce notebook illustre les defis de la conception d'agents autonomes fiables:
- **Controle strict** vs flexibilite
- **Performance** (limiter les requetes) vs exhaustivite
- **Securite** (`allow_dangerously_set_content`) vs protection

Ces compromis sont au coeur de l'ingenierie des systemes multi-agents avec LLMs.